# Capstone Project
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* Introduction: Business Problem
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

## Introduction: Business Problem <a name="introduction"></a>

In the capstone project I will try to find a good location for a Indian restaurant in Manhattan. Specifically, this report will be targeted to stakeholders interested in opening an **Indian restaurant** in **Manhattan**, NY.

Since there are 2874 restaurants in Manhattan I will try to find locations 
* that are not already crowded with restaurants.
* where there are as few Indian restaurants as possible in the closer area around. 
* which are as close to the center of Manhattan as possible. 

With Data Science I will try to find and present to the stakholders the most promissing neigborhoods of Manhattan where to open up a Indian restaurant. 

## Data <a name="data"></a>

Based on the definition of the Business Problem, the decsission will be influenced by the following factors:

* total number of existing restaurants in the neighborhood.
* number of Indian restaurants and the distance to the next one in the neighborhood if there are any. 
* distance of neighborhood from city center.

To find the most promissing neighborhoods to open up a Indian restaurant in Manhattan I will use the following data sources:

* The Information about all the neighborhoods in Manhattan and their centers is available at the **New York Dataset** (https://cocl.us/new_york_dataset).
* To find all restaurants/Indian restaurants in each neighborhood I will use the **Foursquare API**.
* To define a heatmap of the most promissing neighborhoods I need the **borders of each neighborhood** which are available at http://data.beta.nyc//dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/35dd04fb-81b3-479b-a074-a27a37888ce7/download/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson

In [104]:
#!conda install -c conda-forge folium --yes
#!conda install -c conda-forge geopy --yes
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import pyproj
import math

print('All nessecary Libraries imported!')

All nessecary Libraries imported!


Load New York dataset about neigborhoods

In [105]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [106]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [107]:
neighborhoods_data = newyork_data['features']

In [108]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [109]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Filter the dataframe for neighborhoods of Manhattan

In [110]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


get latitude an longitude of manhattan with geopy

In [111]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


create a folium map of New York and mark all Manhattan neigborhoods and the center of Manhattan in it!

In [126]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
folium.CircleMarker([latitude, longitude], radius=7, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_manhattan)    
map_manhattan

Define dataframe with all neighborhoods, latitude, longitude, distance to center of Manhattan, x, y

In [113]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [114]:
#calculate distances from center
distance_from_center=[]
X=[]
Y=[]

manhatten_longitude= longitude
manhatten_latitude=latitude
manhatten_x, manhatten_y= lonlat_to_xy(manhatten_longitude,manhatten_latitude)

for i in range(len(manhattan_data)):
    neigborhood_x, neigborhood_y= lonlat_to_xy(manhattan_data['Longitude'][i],manhattan_data['Latitude'][i])
    distance_from_center.append(calc_xy_distance(manhatten_x, manhatten_y, neigborhood_x, neigborhood_y)) 
    X.append(neigborhood_x)
    Y.append(neigborhood_y)

In [115]:
manhattan_data = manhattan_data.drop('Borough', 1)
manhattan_data['X']=X
manhattan_data['Y']=Y
manhattan_data['Distance from Center']=distance_from_center
manhattan_data

,Neighborhood,Latitude,Longitude,X,Y,Distance from Center
0,Marble Hill,40.876551,-73.910660,-5.794205e+06,9.858099e+06,15945.318731
1,Chinatown,40.715618,-73.994279,-5.821760e+06,9.868103e+06,13386.331413
2,Washington Heights,40.851903,-73.936900,-5.798470e+06,9.861349e+06,10875.558655
3,Inwood,40.867684,-73.921210,-5.795743e+06,9.859410e+06,14045.714502
4,Hamilton Heights,40.823604,-73.949688,-5.803305e+06,9.862859e+06,5825.579136
5,Manhattanville,40.816934,-73.957385,-5.804461e+06,9.863817e+06,4558.839318
6,Central Harlem,40.815976,-73.943211,-5.804573e+06,9.861989e+06,4879.562566
7,East Harlem,40.792249,-73.944182,-5.808594e+06,9.862002e+06,2048.077645
8,Upper East Side,40.775639,-73.960508,-5.811466e+06,9.864025e+06,2450.104944
9,Yorkville,40.775930,-73.947118,-5.811369e+06,9.862302e+06,2904.700389


Insert all Foursquare credetials 

In [116]:
#hidden cell
CLIENT_ID = 'ZZFPNPGKMCMTFXJ03VWM5VB10NGEHUYYFQP3OSKHSAMU5SAS' # your Foursquare ID
CLIENT_SECRET = 'WBXMGAKRY11BE2F5K0RV5VQWGZRNGLIMPPG1XKFJYSNXVCYF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Query all restaurants and all indian  restaurants for each neighborhood from Foursquare API

* food_category = '4d4b7105d754a06374d81259'
* indian_restaurant='4bf58dd8d48988d10f941735'

In [119]:
def getNearbyVenues(names, latitudes, longitudes, category='4bf58dd8d48988d10f941735', radius=500, LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            category,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [118]:
manhattan_restaurants = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [120]:
manhattan_indian_restaurants = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [121]:
print(manhattan_restaurants.shape)
manhattan_restaurants.head(20)

(2874, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner
2,Marble Hill,40.876551,-73.910660,Dunkin',40.877136,-73.906666,Donut Shop
3,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
4,Marble Hill,40.876551,-73.910660,Parrilla Latina,40.877473,-73.906073,Steakhouse
5,Marble Hill,40.876551,-73.910660,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
6,Marble Hill,40.876551,-73.910660,Boston Market,40.877430,-73.905412,American Restaurant
7,Marble Hill,40.876551,-73.910660,SUBWAY,40.878493,-73.905385,Sandwich Place
8,Marble Hill,40.876551,-73.910660,Subway,40.877720,-73.905380,Sandwich Place
9,Marble Hill,40.876551,-73.910660,Hernandez Grocery,40.875897,-73.912591,Deli / Bodega


In [122]:
print(manhattan_indian_restaurants.shape)
manhattan_indian_restaurants.head(20)

(274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Nyonya,40.719155,-73.996893,Malay Restaurant
1,Chinatown,40.715618,-73.994279,New Malaysia,40.715787,-73.996905,Malay Restaurant
2,Chinatown,40.715618,-73.994279,Dirt Candy,40.717890,-73.991015,Vegetarian / Vegan Restaurant
3,Chinatown,40.715618,-73.994279,Sanuria Restaurant,40.714681,-73.998006,Malay Restaurant
4,Chinatown,40.715618,-73.994279,Curry House Indian Cuisine,40.719046,-73.990849,Indian Restaurant
5,Chinatown,40.715618,-73.994279,Roasting Plant Coffee,40.717784,-73.990453,Coffee Shop
6,Washington Heights,40.851903,-73.936900,Kismat Indian Restaurant,40.855222,-73.936967,Indian Restaurant
7,Hamilton Heights,40.823604,-73.949688,Clove Indian Restaurant & Bar,40.821280,-73.950620,Indian Restaurant
8,Hamilton Heights,40.823604,-73.949688,Mumbai Masala,40.826866,-73.946486,Indian Restaurant
9,Manhattanville,40.816934,-73.957385,Chapati House - NYC,40.814572,-73.959154,Indian Restaurant


In [123]:
print('Total number of restaurants in Manhattan:', len(manhattan_restaurants))
print('Total number of Indian restaurants in Manhatten:', len(manhattan_indian_restaurants))
print('Percentage of Indian restaurants in Mahattan: {:.2f}%'.format(len(manhattan_indian_restaurants) / len(manhattan_restaurants) * 100))

Total number of restaurants in Manhattan: 2874
Total number of Indian restaurants in Manhatten: 274
Percentage of Indian restaurants in Mahattan: 9.53%


Create a folium map to display all restaurants in Manhatten and show them in different colors. **Indian restauants in blue** and **other restauratns in red** and the **center of Manhattan in orange**

In [125]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.CircleMarker([latitude, longitude], radius=7, color='orange', fill=True, fill_color='orange', fill_opacity=1).add_to(map_manhattan)  
for lat, lng in zip(manhattan_restaurants['Venue Latitude'], manhattan_restaurants['Venue Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
for lat, lng in zip(manhattan_indian_restaurants['Venue Latitude'], manhattan_indian_restaurants['Venue Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
map_manhattan

Now we have gatherd all the data we need to find the most promissing neighborhoods in Manhatten to open up a Indian restaurant.

* We have all the Data about the neighborhoods, we know the center of each neighborhood
* We have all the restaurants in manhatten, we know in which neighborhood and where exactly they are located and we know what type of restaurant it is. 
* We know the location of the center of Manhattan and the distance of each neighborhood to the center.  

This concludes the Data preparation phase and now we can continue with the analysis of the data to find the most promissing neighborhoods. 

In [ ]:
map_manhattan
map_berlin = folium.Map(location=berlin_center, zoom_start=13)
folium.Marker(berlin_center, popup='Alexanderplatz').add_to(map_manhattan)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_berlin)
map_berlin

In [240]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.CircleMarker([latitude, longitude], radius=7, color='orange', fill=True, fill_color='orange', fill_opacity=1).add_to(map_manhattan)  



for lat, lng, label, indian in zip(restaurants_df['Venue Latitude'], restaurants_df['Venue Longitude'], restaurants_df['Venue'], restaurants_df['_merge']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue' if indian=='both' else 'red',
        fill=True,
        fill_color='color',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
map_manhattan


for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_berlin)
map_berlin

In [342]:
restaurants_df.values()

TypeError: 'numpy.ndarray' object is not callable

In [328]:
restaurants_df=pd.merge(manhattan_restaurants, manhattan_indian_restaurants, how='left', indicator=True)
restaurants_df.shape
restaurants_df.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,_merge
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place,left_only
1,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner,left_only
2,Marble Hill,40.876551,-73.910660,Dunkin',40.877136,-73.906666,Donut Shop,left_only
3,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant,left_only
4,Marble Hill,40.876551,-73.910660,Parrilla Latina,40.877473,-73.906073,Steakhouse,left_only
5,Marble Hill,40.876551,-73.910660,Subway Sandwiches,40.874667,-73.909586,Sandwich Place,left_only
6,Marble Hill,40.876551,-73.910660,Boston Market,40.877430,-73.905412,American Restaurant,left_only
7,Marble Hill,40.876551,-73.910660,SUBWAY,40.878493,-73.905385,Sandwich Place,left_only
8,Marble Hill,40.876551,-73.910660,Subway,40.877720,-73.905380,Sandwich Place,left_only
9,Marble Hill,40.876551,-73.910660,Hernandez Grocery,40.875897,-73.912591,Deli / Bodega,left_only


In [326]:
manhattan_restaurants.shape

(2874, 7)

In [100]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


In [ ]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [101]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 337 uniques categories.


In [102]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.010000,...,0.00,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000,0.030000
2,Central Harlem,0.000000,0.00,0.00,0.065217,0.043478,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.000000,0.030000,0.01,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.010000,0.000000
4,Chinatown,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.000000,0.00,0.00,0.000000,0.030000,0.01,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.010000,0.020000,0.010000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.020000,0.030000,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.000000,0.020000,0.01,0.00,0.020000,0.010000,...,0.00,0.00,0.000000,0.00,0.00,0.050000,0.020000,0.000000,0.000000,0.000000
9,Financial District,0.010000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000,0.000000
